In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#https://scipython.com/blog/quadtrees-2-implementation-in-python/
#https://pydoc.net/openslide-python/1.1.1/openslide/

import numpy as np
import openslide
from probreg import cpd
from probreg import transformation as tf
import cv2
from PIL import Image
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import json

import pickle

In [3]:
## HACK get rid of it
import sys
sys.path.append("..")
from registration_tree import Rect, QuadTree

## Unscharf:

- Z_BB_569923_1 
- Z_BB_574882_1
- Z_BB_570370_1
- Z_BB_566933L_1

In [4]:
source_slide_paths = list(Path(r"D:\Datasets\ScannerStudy\Aperio\Cyto").glob("*.svs"))

len(source_slide_paths)

5

In [5]:
target_slide_paths = list(Path(r"D:\Datasets\ScannerStudy\NanoZoomer2.0HT\Cyto").glob("*.ndpi")) 

len(target_slide_paths)

5

In [6]:
source_slide_paths = {name.stem.split("_")[2].split("_")[0] : 
                      {"name": name, "slide": openslide.OpenSlide(str(name))}
                      for name in source_slide_paths}
source_slide_paths.keys()

dict_keys(['563476', '563479', '568320', '568381', '574162'])

In [7]:
target_slide_paths = {name.stem.split("_")[2].split("_")[0] : 
                      {"name": name, "slide": openslide.OpenSlide(str(name))}
                      for name in target_slide_paths}
target_slide_paths.keys()

dict_keys(['563476', '563479', '568320', '568381', '574162'])

In [8]:
for name in source_slide_paths:
    if name not in target_slide_paths:
        print(name)

In [9]:
for name in target_slide_paths:
    if name not in source_slide_paths:
        print(name)

In [10]:
df = pd.read_pickle("../../../SDATA/Statistics/SDATA_Final_Annotations.pkl")
df = df[df["species"] == "equine"]

df["x1"] = [vector['x1'] if type(vector) is dict else 0 for vector in df["vector"]]
df["y1"] = [vector['y1'] if type(vector) is dict else 0 for vector in df["vector"]]

df["x2"] = [vector['x2'] if type(vector) is dict else 0 for vector in df["vector"]]
df["y2"] = [vector['y2'] if type(vector) is dict else 0 for vector in df["vector"]]

df["center_x"] = [x1 + ((x2-x1) / 2) for x1, x2 in zip(df["x1"], df["x2"])]
df["center_y"] = [y1 + ((y2-y1) / 2) for y1, y2 in zip(df["y1"], df["y2"])]

df["anno_width"] = [x2-x1 for x1, x2 in zip(df["x1"], df["x2"])]
df["anno_height"]= [y2-y1 for y1, y2 in zip(df["y1"], df["y2"])]

df.head()

,id,image_id,image_set,species,image_name,image_type,grade,vector,user_id,deleted,...,data_set_name,version,x1,y1,x2,y2,center_x,center_y,anno_width,anno_height
unique_identifier,,,,,,,,,,,,,,,,,,,,,
797adcd7-aeb7-4bb1-bceb-2ee094f91c39,2577504,5232,268,equine,19_EIPH_566933 L Berliner Blau.svs,Prussian,0,"{'x1': 22302, 'x2': 22386, 'y1': 15891, 'y2': ...",1,False,...,SDATA,Density,22302,15891,22386,15975,22344.0,15933.0,84,84
ecf68944-5bf6-44c2-8f01-baa23470939c,2577505,5232,268,equine,19_EIPH_566933 L Berliner Blau.svs,Prussian,0,"{'x1': 26391, 'x2': 26480, 'y1': 21617, 'y2': ...",1,False,...,SDATA,Density,26391,21617,26480,21706,26435.5,21661.5,89,89
0c00d34b-8ad9-43fc-b182-e2afb48204c8,2577506,5232,268,equine,19_EIPH_566933 L Berliner Blau.svs,Prussian,0,"{'x1': 30377, 'x2': 30459, 'y1': 12843, 'y2': ...",1,False,...,SDATA,Density,30377,12843,30459,12925,30418.0,12884.0,82,82
5a980dfd-c77a-4a64-b874-dfb6f0e04285,2577507,5232,268,equine,19_EIPH_566933 L Berliner Blau.svs,Prussian,0,"{'x1': 13150, 'x2': 13253, 'y1': 22302, 'y2': ...",1,False,...,SDATA,Density,13150,22302,13253,22406,13201.5,22354.0,103,104
0d28d4d1-e53e-40fc-88f3-0553d62007e8,2577508,5232,268,equine,19_EIPH_566933 L Berliner Blau.svs,Prussian,0,"{'x1': 12288, 'x2': 12338, 'y1': 6693, 'y2': 6...",1,False,...,SDATA,Density,12288,6693,12338,6785,12313.0,6739.0,50,92


In [11]:
%%time

results = {}
#target_depth = 2

for thumbnail_size in [(1024, 1024), (2048, 2048), (4096, 4096)]:
    
    for target_depth in [0, 1, 2]:
        
        result_folder = Path(f"Depth_{target_depth}/{thumbnail_size[0]}")

        result_folder.mkdir(parents=True, exist_ok=True)
    
        for name, source in source_slide_paths.items():

            parameter = {
                # feature extractor parameters
                "point_extractor": "sift",  #orb , sift
                "maxFeatures": 512, 
                "crossCheck": False, 
                "flann": False,
                "ratio": 0.5, 
                "use_gray": False,

                # QTree parameter 
                "homography": True,
                "filter_outliner": False,
                "debug": True,
                "target_depth": target_depth,
                "run_async": False,
                "thumbnail_size": thumbnail_size
            }

            target = target_slide_paths[name]


            source_slide = source["slide"]
            target_slide = target["slide"]

            source_dimension = Rect.create(Rect, 0, 0, source_slide.dimensions[0], source_slide.dimensions[1])
            target_dimension = Rect.create(Rect, 0, 0, target_slide.dimensions[0], target_slide.dimensions[1])

            qtree = QuadTree(source_dimension, source_slide, target_dimension, target_slide, **parameter)

            results[source["name"]] = qtree
            
            with open(result_folder / f'{qtree.source_name}-To-{qtree.target_name}.pickle', 'wb') as handle:
                pickle.dump(qtree, handle, protocol=pickle.HIGHEST_PROTOCOL)

Wall time: 55min 50s


In [ ]:
for path, qtree in results.items():
    
    df_gt = df[df["image_name"].str.contains(path.stem.split("_")[-2])].sort_values(by=["grade"], ascending=False)
    
    annos = np.array(df_gt[["center_x", "center_y", "anno_width", "anno_height"]]) 
    qtree.draw_annotations(annos, num_sub_pic=5, figsize=(10, 10))

..\registration_tree.py:442: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(constrained_layout=True, figsize=figsize)
